# Udacity Data Engineer - US Airports and Immigration Data Integration and ETL Data Pipeline

### Data Engineering Capstone Project

#### Project Summary

The main aim of this project is to integrate I94 immigration data, world temperature data and US demographic data with ETL data pipelines to build the a data warehouse with fact and dimension tables to derive and generate the data insights for the business reporting.


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

###  Step 0 : Imporing the Required Python Libraries and Modules

In [1]:
# Importing the Python Libraries and Modules e.g. Pandas, PySpark etc.

import pandas as pd
import os
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col, to_timestamp
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear,dayofweek
from pyspark.sql.types import *

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [2]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


#####  Listing Down files from current working directory

In [3]:
#cwd = os.getcwd()

cwd = os.listdir()
print("Current working directory:", cwd)

Current working directory: ['.ipynb_checkpoints', 'airport-codes_csv.csv', 'Capstone Project Template.ipynb', 'GlobalLandTemperaturesByCity.csv', 'I94_SAS_Labels_Descriptions.SAS', 'immigration_data_sample.csv', 'udacity-capstone-project (1).ipynb', 'Udacity_Capstone_Data_Engineer_Nanodegree_Aseem_Narula.ipynb', 'us-cities-demographics.csv', 'US_Immigration_Data_Lake.ipynb']


### Step 1: Scope the Project and Gather Data

#### Scope 
The main aim of this project is to integrate I94 immigration data, world temperature data and US demographic data with ETL data pipelines to build the a data warehouse with fact and dimension tables to derive and generate the data insights for the business reporting.

#### Describe and Gather Data 
The datasets are the input flat files majorly in the CSV, SAS file formats from various sources,we will be first analysing the datasets followed by clearing the source data.


#### Datasets
The following datasets are included in the project workspace. 

##### I94 Immigration Data: 

This data comes from the US National Tourism and Trade Office. Each report contains international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only), and the top ports of entry (for select countries). https://travel.trade.gov/research/reports/i94/historical/2016.html

##### World Temperature Data: 
This dataset came from Kaggle. https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data

##### U.S. City Demographic Data: 
This data comes from OpenSoft. https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/

##### Airport Code Table: 
This is a simple table of airport codes and corresponding cities. https://datahub.io/core/airport-codes#data

##### Reading the Immigration Data in Dataframe

In [4]:
# Reading the Immigration Data in the Dataframe

df_immigration_data = pd.read_csv("immigration_data_sample.csv")

In [5]:
# Displaying rows to preview the immigration data

df_immigration_data.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


##### Reading the Global Land Temparature by City Data in the Dataframe

In [6]:
# Reading the Global Temparature by City Data in the Dataframe

df_temperature_data = pd.read_csv("GlobalLandTemperaturesByCity.csv")

In [7]:
# Displaying rows to preview the Global Temparature by City Data

df_temperature_data.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


##### Reading the U.S. City Demographic Data in the Dataframe

In [8]:
# Reading the US City Demographic Data in the Dataframe

df_demographics_data = pd.read_csv("us-cities-demographics.csv",delimiter=';')

In [9]:
# Previewing the Demo Graphics Data

df_demographics_data.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


##### Reading the Airports Code Data in the Dataframe

In [10]:
# Reading the Airports Code Data in the Dataframe

df_airports_code_data = pd.read_csv("airport-codes_csv.csv")

In [11]:
# Prevewing the rows for Airports Code Data from Dataframe

df_airports_code_data.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Step 2: Explore and Assess the Data

In this step, we will explore the dataset for their data quality, finding null values, finding the duplicates, filling the NaN values followed by cleaning, writing code for the data wrangling and definining the data quality checks etc.

We will also explore the data dictioanry for the datasets

#### Step 2(a) :Explore the data 



In [12]:
# Printing the shape of the dataframes

print("Shape of df_immigration_data:", df_immigration_data.shape)
print("Shape of df_temperature_data:", df_temperature_data.shape)
print("Shape of df_demographics_data:", df_demographics_data.shape)
print("Shape of df_airports_code_data:", df_airports_code_data.shape)


Shape of df_immigration_data: (1000, 29)
Shape of df_temperature_data: (8599212, 7)
Shape of df_demographics_data: (2891, 12)
Shape of df_airports_code_data: (55075, 12)


In [13]:
# Number of Rows and Columns in Immigration data

print("Number of Rows in df_immigration_data:", len(df_immigration_data))
print("Number of Columns in df_immigration_data:", len(df_immigration_data.columns))

Number of Rows in df_immigration_data: 1000
Number of Columns in df_immigration_data: 29


In [14]:
# Number of Rows and Columns in Global Temperature data

print("Number of Rows in df_temperature_data:", len(df_temperature_data))
print("Number of Columns in df_temperature_data:", len(df_temperature_data.columns))

Number of Rows in df_temperature_data: 8599212
Number of Columns in df_temperature_data: 7


In [15]:
# Number of Rows and Columns in US Demographics data

print("Number of Rows in df_demographics_data:", len(df_demographics_data))
print("Number of Columns in df_demographics_data:", len(df_demographics_data.columns))

Number of Rows in df_demographics_data: 2891
Number of Columns in df_demographics_data: 12


In [16]:
# Number of Rows and Columns in Airports Code data

print("Number of Rows in df_airports_code_data:", len(df_airports_code_data))
print("Number of Columns in df_airports_code_data:", len(df_airports_code_data.columns))

Number of Rows in df_airports_code_data: 55075
Number of Columns in df_airports_code_data: 12


In [17]:
# Size of the dataframes

print("Number of Elements in df_immigration_data:", df_immigration_data.size)
print("Number of Elements in df_temperature_data:", df_temperature_data.size)
print("Number of Elements in df_demographics_data:", df_demographics_data.size)
print("Number of Elements in df_airports_code_data:", df_airports_code_data.size)

Number of Elements in df_immigration_data: 29000
Number of Elements in df_temperature_data: 60194484
Number of Elements in df_demographics_data: 34692
Number of Elements in df_airports_code_data: 660900


In [18]:
# Providing the concise summary of the DataFrames, including the number of non-null values and data types for each column

# Data Dictionary - i.e. type of the data types in the each dataset

# Immigration I94 --- List of columns

df_immigration_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
Unnamed: 0    1000 non-null int64
cicid         1000 non-null float64
i94yr         1000 non-null float64
i94mon        1000 non-null float64
i94cit        1000 non-null float64
i94res        1000 non-null float64
i94port       1000 non-null object
arrdate       1000 non-null float64
i94mode       1000 non-null float64
i94addr       941 non-null object
depdate       951 non-null float64
i94bir        1000 non-null float64
i94visa       1000 non-null float64
count         1000 non-null float64
dtadfile      1000 non-null int64
visapost      382 non-null object
occup         4 non-null object
entdepa       1000 non-null object
entdepd       954 non-null object
entdepu       0 non-null float64
matflag       954 non-null object
biryear       1000 non-null float64
dtaddto       1000 non-null object
gender        859 non-null object
insnum        35 non-null float64
airline       967 non

In [19]:
# Global Temperature Data - List of Columns

df_temperature_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [20]:
# US Demographics Data -List of Columns

df_demographics_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.2+ KB


In [21]:
# Airports code data - List of Columns

df_airports_code_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [22]:
# Describe Method ----> Generates descriptive statistics of the DataFrame, including count, mean, std, min, and max.

# Immigration I94 DataSet

df_immigration_data.describe()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,dtadfile,entdepu,biryear,insnum,admnum
count,1.000000e+03,1.000000e+03,1000.0,1000.0,1000.000000,1000.00000,1000.000000,1000.000000,951.000000,1000.000000,1000.000000,1000.0,1.000000e+03,0.0,1000.000000,35.000000,1.000000e+03
mean,1.542097e+06,3.040461e+06,2016.0,4.0,302.928000,298.26200,20559.680000,1.078000,20575.037855,42.382000,1.859000,1.0,2.016042e+07,NaN,1973.618000,3826.857143,6.937237e+10
std,9.152879e+05,1.799818e+06,0.0,0.0,206.485285,202.12039,8.995027,0.485955,24.211234,17.903424,0.386353,0.0,4.951657e+01,NaN,17.903424,221.742583,2.338134e+10
min,1.092500e+04,1.320800e+04,2016.0,4.0,103.000000,103.00000,20545.000000,1.000000,20547.000000,1.000000,1.000000,1.0,2.016040e+07,NaN,1923.000000,3468.000000,0.000000e+00
25%,7.214422e+05,1.412170e+06,2016.0,4.0,135.000000,131.00000,20552.000000,1.000000,20561.000000,30.750000,2.000000,1.0,2.016041e+07,NaN,1961.000000,3668.000000,5.599301e+10
50%,1.494568e+06,2.941176e+06,2016.0,4.0,213.000000,213.00000,20560.000000,1.000000,20570.000000,42.000000,2.000000,1.0,2.016042e+07,NaN,1974.000000,3887.000000,5.931477e+10
75%,2.360901e+06,4.694151e+06,2016.0,4.0,438.000000,438.00000,20567.250000,1.000000,20580.000000,55.000000,2.000000,1.0,2.016042e+07,NaN,1985.250000,3943.000000,9.343623e+10
max,3.095749e+06,6.061994e+06,2016.0,4.0,746.000000,696.00000,20574.000000,9.000000,20715.000000,93.000000,3.000000,1.0,2.016080e+07,NaN,2015.000000,4686.000000,9.502151e+10


In [23]:
# Global Temperature Dataset Statiscs

df_temperature_data.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,8.235082e+06,8.235082e+06
mean,1.672743e+01,1.028575e+00
std,1.035344e+01,1.129733e+00
min,-4.270400e+01,3.400000e-02
25%,1.029900e+01,3.370000e-01
50%,1.883100e+01,5.910000e-01
75%,2.521000e+01,1.349000e+00
max,3.965100e+01,1.539600e+01


In [24]:
# US Demographics Dataset Statistics

df_demographics_data.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


In [25]:
# Airports Code Dataset Statistics

df_airports_code_data.describe()

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


#### Step 2(b) : Asses the data and perform data quality checks including identify nulls, NaNs, finding duplicates etc.

In [26]:
# Finding the duplicates for the Immigration I94 data

df_immigration_data_dupes_check = df_immigration_data[df_immigration_data.duplicated(subset=None, keep='first')]

#### There are no duplicate values in the Immigration Data Frame

In [27]:
# Checking the results of the duplicates values in the Immigration Dataframe.

df_immigration_data_dupes_check

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype


In [28]:
# Finding the nulls for the Immigration I94 data, count the total number of null values in each column

df_missing_values_immigration_data = df_immigration_data.isnull().sum()

# Display the count of the sum result

print("Following columns are having the NULL values in the dataset: \n", df_missing_values_immigration_data)

Following columns are having the NULL values in the dataset: 
 Unnamed: 0       0
cicid            0
i94yr            0
i94mon           0
i94cit           0
i94res           0
i94port          0
arrdate          0
i94mode          0
i94addr         59
depdate         49
i94bir           0
i94visa          0
count            0
dtadfile         0
visapost       618
occup          996
entdepa          0
entdepd         46
entdepu       1000
matflag         46
biryear          0
dtaddto          0
gender         141
insnum         965
airline         33
admnum           0
fltno            8
visatype         0
dtype: int64


In [29]:
# Finding the nulls for the US Demographics Dataset data, count the total number of null values in each column

df_missing_values_demographics_data = df_demographics_data.isnull().sum()

# Display the count of the sum result

print("Following columns are having the NULL values in the dataset: \n", df_missing_values_demographics_data)

Following columns are having the NULL values in the dataset: 
 City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64


In [30]:
# Finding the nulls for the Gloabl Temperature Dataset, count the total number of null values in each column

df_missing_values_temperature_data = df_temperature_data.isnull().sum()

# Display the count of the sum result

print("Following columns are having the NULL values in the dataset: \n", df_missing_values_temperature_data)

Following columns are having the NULL values in the dataset: 
 dt                                    0
AverageTemperature               364130
AverageTemperatureUncertainty    364130
City                                  0
Country                               0
Latitude                              0
Longitude                             0
dtype: int64


In [31]:
# Finding the nulls for the Airports Code Dataset data, count the total number of null values in each column

df_missing_values_airports_data = df_airports_code_data.isnull().sum()

# Display the count of the sum result

print("Following columns are having the NULL values in the dataset: \n", df_missing_values_airports_data)

Following columns are having the NULL values in the dataset: 
 ident               0
type                0
name                0
elevation_ft     7006
continent       27719
iso_country       247
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64


### Step 3: Cleaning the dataset and define the Data Model for STAR Schema with data dictionary details for each table

In this third step of ETL journey, we will be cleaning the datasets, defining the data model for STAR schema with data dictionary details for each table, the pre-processing of the Immigration data followed by the creating I94 fact table surronded by the various dimension tables to fulfill the Data Warehousing Business requirement which will be used for meaningful insights, data visualization and business reporting for the stakeholders and the strategic data based driven decisions.


#### Step 3(a) : Detailing Data Dictionary 

#### Data Dictionary

Columns descriptions - Immigration Table

| Column Name | Description |
| :--- | :--- |
| CICID | ID that uniquely identify one record in the dataset |
| I94YR | 4 digit year |
| I94MON | Numeric month |
| I94CIT | 3 digit code of source city for immigration (Born country) |
| I94RES | 3 digit code of source country for immigration (Residence country) |
| I94PORT | Port addmitted through |
| ARRDATE | Arrival date in the USA |
| I94MODE | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) |
| I94ADDR | State of arrival |
| DEPDATE | Departure date from the USA |
| I94BIR | Age of Respondent in Years |
| I94VISA | Visa codes collapsed into three categories: (1 = Business; 2 = Pleasure; 3 = Student) |
| COUNT | Used for summary statistics |
| DTADFILE | Character Date Field |
| VISAPOST | Department of State where where Visa was issued |
| OCCUP | Occupation that will be performed in U.S. |
| ENTDEPA | Arrival Flag. Whether admitted or paroled into the US |
| ENTDEPD | Departure Flag. Whether departed, lost visa, or deceased |
| ENTDEPU | Update Flag. Update of visa, either apprehended, overstayed, or updated to PR |
| MATFLAG | Match flag |
| BIRYEAR | 4 digit year of birth |
| DTADDTO | Character date field to when admitted in the US (allowed to stay until) |
| GENDER | Gender |
| INSNUM | INS number |
| AIRLINE | Airline used to arrive in U.S. |
| ADMNUM | Admission number, should be unique and not nullable |
| FLTNO | Flight number of Airline used to arrive in U.S. |
| VISATYPE | Class of admission legally admitting the non-immigrant to temporarily stay in U.S. |

Columns Descritpion - Global Temperature 

| Column Name | Description |
| :--- | :--- |
| dt | Date in format YYYY-MM-DD |
| AverageTemperature | Average temperature of the city in a given date |
| City | City Name |
| Country | Country Name |
| Latitude | Latitude |
| Longitude | Longitude |

Columns Description - Airport Code Table


| Column Name | Description |
| :--- | :--- |
| ident | Unique identifier |
| type | Type of the airport |
| name | Airport Name |
| elevation_ft | Altitude of the airport |
| continent | Continent |
| iso_country | ISO code of the country of the airport |
| iso_region | ISO code for the region of the airport |
| municipality | City where the airport is located |
| gps_code | GPS code of the airport |
| iata_code | IATA code of the airport |
| local_code | Local code of the airport |
| coordinates | GPS coordinates of the airport |

#### Step 3(b) : Cleaning the data sets

In [50]:
# Extracting the data columns for the FACT Immigration table 
fact_immigration_table = df_immigration_data[['cicid', 'i94yr', 'i94mon', 'i94port', 'i94addr', 'arrdate', 'depdate', 'i94mode', 'i94visa']]
fact_immigration_table.columns = ['cic_id', 'year', 'month', 'city_code', 'state_code', 'arrive_date', 'departure_date', 'mode', 'visa']
fact_immigration_table.loc[:, 'country'] = 'United States'
fact_immigration_table.head(8)

,cic_id,year,month,city_code,state_code,arrive_date,departure_date,mode,visa,country
0,4084316.0,2016.0,4.0,HHW,HI,20566.0,20573.0,1.0,2.0,United States
1,4422636.0,2016.0,4.0,MCA,TX,20567.0,20568.0,1.0,2.0,United States
2,1195600.0,2016.0,4.0,OGG,FL,20551.0,20571.0,1.0,2.0,United States
3,5291768.0,2016.0,4.0,LOS,CA,20572.0,20581.0,1.0,2.0,United States
4,985523.0,2016.0,4.0,CHM,NY,20550.0,20553.0,3.0,2.0,United States
5,1481650.0,2016.0,4.0,ATL,GA,20552.0,20606.0,1.0,2.0,United States
6,2197173.0,2016.0,4.0,SFR,CA,20556.0,20635.0,1.0,2.0,United States
7,232708.0,2016.0,4.0,NYC,NY,20546.0,20554.0,1.0,2.0,United States


In [60]:
# Extracting data for the DIMENSION Immigration Person Table
dimension_immigration_person_table = df_immigration_data[['cicid', 'i94cit', 'i94res', 'biryear', 'gender', 'insnum']]
dimension_immigration_person_table.columns = [['cic_id', 'citizen_country', 'residence_country', 'birth_year', 'gender', 'ins_number']]
dimension_immigration_person_table.head(8)

,cic_id,citizen_country,residence_country,birth_year,gender,ins_number
0,4084316.0,209.0,209.0,1955.0,F,NaN
1,4422636.0,582.0,582.0,1990.0,M,NaN
2,1195600.0,148.0,112.0,1940.0,M,NaN
3,5291768.0,297.0,297.0,1991.0,M,NaN
4,985523.0,111.0,111.0,1997.0,F,NaN
5,1481650.0,577.0,577.0,1965.0,M,NaN
6,2197173.0,245.0,245.0,1968.0,F,NaN
7,232708.0,113.0,135.0,1983.0,F,NaN


In [62]:
# Extracting data for the DIMENSION Immigration Airlines Table
dimension_immigration_airlines_table = df_immigration_data[['cicid', 'airline', 'admnum', 'fltno', 'visatype']]
dimension_immigration_airlines_table.columns = ['cic_id', 'airline', 'admin_num', 'flight_number', 'visa_type']
dimension_immigration_airlines_table.head(10)

,cic_id,airline,admin_num,flight_number,visa_type
0,4084316.0,JL,5.658267e+10,00782,WT
1,4422636.0,*GA,9.436200e+10,XBLNG,B2
2,1195600.0,LH,5.578047e+10,00464,WT
3,5291768.0,QR,9.478970e+10,00739,B2
4,985523.0,NaN,4.232257e+10,LAND,WT
5,1481650.0,DL,7.368526e+08,910,B2
6,2197173.0,CX,7.863122e+08,870,B2
7,232708.0,BA,5.547449e+10,00117,WT
8,5227851.0,LX,5.941342e+10,00008,WT
9,13213.0,AA,5.544979e+10,00109,WT


In [79]:
# Extracting data for the DIMENSION Immigration Temperature Table
                                                  
dimension_immigration_temperature_table = df_temperature_data
#dimension_immigration_temperature_table = df_temperature_data[df_temperature_data['Country'] == 'United States']
dimension_immigration_temperature_table = dimension_immigration_temperature_table[['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City', 'Country','Latitude','Longitude']]
dimension_immigration_temperature_table.columns = ['temperature_date', 'average_temperature', 'average_temperature_uncertainty', 'city', 'country','latitude','longitude']

dimension_immigration_temperature_table.head(8)

,temperature_date,average_temperature,average_temperature_uncertainty,city,country,latitude,longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E


In [80]:
# Extracting data for the DIMENSION Average Temperature Table

dimension_average_usa_temperature_table = dimension_immigration_temperature_table.groupby(["country"]).agg({"average_temperature": "mean", 
                                                                                                            "latitude": "first", "longitude": "first"}).reset_index().reset_index()
dimension_average_usa_temperature_table.head()

,index,country,average_temperature,latitude,longitude
0,0,Afghanistan,13.816497,36.17N,69.61E
1,1,Albania,15.525828,40.99N,19.17E
2,2,Algeria,17.763206,36.17N,3.98E
3,3,Angola,21.759716,12.05S,13.15E
4,4,Argentina,16.999216,39.38S,62.43W
